#  DFFRAM Compiler


 Standard Cell Library based Memory Compiler using DFF/Latch cells.





# Overview: 
DFFRAM is based around two Python modules: the DFFRAM flow and the placeram utility.

placeram is a custom placer using OpenROAD's Python interface. It places DFFRAM RAM/register file designs in a predetermined structure to avoid a lengthy and inefficient manual placement process for RAM. 

dffram.py is a relatively self-contained flow that uses placeram and OpenLane to place, route and harden RAM. 



---



Installing Conda Package Manager for Google Colab

In [1]:
!pip install -q condacolab
import condacolab

condacolab.install_from_url(
    "https://repo.anaconda.com/miniconda/Miniconda3-py37_4.11.0-Linux-x86_64.sh"
)

In [ ]:
import condacolab

condacolab.check()
!sed -i -e /cudatoolkit/d /usr/local/conda-meta/pinned

The litex-hub conda repository maintains a collection of conda packages recipes that are used by DFFRAM, namely: 

*  Magic 
*  Openroad 
*  Netgen
*  Yosys 
*  Gdstk

In [ ]:
!conda install -y -c litex-hub -c conda-forge open_pdks.sky130a=1.0.290 magic openroad netgen yosys gdstk

Cloning OpenLane and DFFRAM and Installing the required dependencies:  🔄

(We're also setting some environment variables that make OpenLane and DFFRAM play nicer with conda.)

In [ ]:
%cd /content

!apt-get update && apt-get install -yq tcllib
%env TCLLIBPATH=/usr/share/tcltk

!rm -rf ./openlane
!git clone --depth 1 https://github.com/The-OpenROAD-Project/OpenLane openlane
!python3 -m pip install -r /content/openlane/requirements.txt

!rm -rf ./DFFRAM
!git clone --depth 1 https://github.com/Cloud-V/DFFRAM dffram
%env NO_CHECK_INSTALL=1

%cd /content/dffram



---



# Invoke for a full list of different options 🤔  

In [ ]:
!./dffram.py --help

# Supported Sizes: 

RAM: 
- 32 words with byte write enable (1RW and 1RW1R).
- 128 words with byte write enable (1RW and 1RW1R).
- 256 words with byte write enable (1RW and 1RW1R).
- 512 words with byte write enable (1RW and 1RW1R).
- 1024 words with byte write enable (1RW and 1RW1R).
- 2048 words with byte write enable (1RW and 1RW1R).


RF: 

- 32 x 32-bit words (2R1W)

# 32x32 with one read-write port and one read-only port 🚀 

In [ ]:
!./dffram.py --using-local-openlane /content/openlane --pdk-root /usr/local/share/pdk -s 32x32

# 32x32 with two read ports and one write port (a register file)

In [ ]:
!./dffram.py --using-local-openlane /content/openlane --pdk-root /usr/local/share/pdk -s 32x32 -b ::rf -v 2R1W

# Check it out 🎉

In [ ]:
import pathlib
import gdstk
from IPython.display import SVG

gds = sorted(
    pathlib.Path("/content/DFFRAM/build/32x32_DEFAULT").glob("*/gds/RAM32.gds")
)[0]
library = gdstk.read_gds(gds)
top_cells = library.top_level()
top_cells[0].write_svg("RAM32.svg")
SVG("RAM32.svg")

# The Secret Menu! 🤫

DFFRAM supports a number of secret options you can use to further customize your experience. They are all passed as environment variables.

Though do note using these variables is very likely to break things.


| Environment Variable | Effect |
| - | - |
| FORCE_ACCEPT_SIZE | DFFRAM checks that you are not using a size not officially marked supported as available by a certain building block set. If this environment variable is set to any value, the check is bypassed. |
| FORCE_DESIGN_NAME | Design names are found based on the size. If you'd like to force dffram to use a specific design name instead, set this environment variable to that name. |